- 코드 실행 전 터미널에서 설치  
    https://github.com/noorkhokhar99/face-detection-yolov8/tree/main  
    git clone https://github.com/noorkhokhar99/face-detection-yolov8.git  
    cd face-detection-yolov8  
    pip install ultralytics==8.0.0  

    
- 아래 두줄은 잘 안됨. 그래서 아래 코드셀로 실행함.  
    imagefile  
    python filename.py  

- RetinaFace_Resnet => YOLOv8 순으로 진행하였음. 
- retina는 인식할 얼굴이 없으면 결과의 타입이 달라짐. ','가 문제가 되는 거 같음.   
    => 이것을 해결하기 위해 try, except를 사용. 실행 순서 상관없어짐.
- YOLOv8에서 인식못하는 몇몇을  RetinaFace 에서 인식할 수 있음.
- 그러나 RetinaFace는 3.5시간/2500장, YOLOv8은 40분/2500장


In [10]:
from ultralytics import YOLO
import numpy as np
import cv2
import os

In [11]:
model = YOLO('./face-detection-yolov8/yolov8n-face.pt')
# yolov8n.pt for object detection

# check the shape of outs

In [8]:
image_path = 'C:/Users/pc3/Desktop/unidentify_10000ea/KAMO/ident_image/rear/1681710996530154943.png'
outs = model.predict(
                source=image_path,
                conf=0.1,
                save=True,
                imgsz=1280,
                hide_labels=True,
                max_det=1000,
                show=False,
                )

# 더 많은 것을 인식하기 위해서는 conf 값을 줄이면 됨
outs
# outs.ndim
np_array = np.array([tensor.numpy() for tensor in outs])
# outs, np_array.shape[1]
# tessor에 갖힌 list
# max_det : 최대 인식 건수

Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\KAMO\ident_image\rear\1681710996530154943.png: 736x1280 5 faces, 488.0ms
Speed: 6.0ms pre-process, 488.0ms inference, 3.0ms postprocess per image at shape (1, 3, 1280, 1280)
Results saved to runs\detect\predict2


In [9]:
np_array

array([[[        656,         213,         690,         253,     0.79068,           0],
        [        765,         224,         796,         264,     0.77297,           0],
        [        941,         190,         977,         229,     0.67244,           0],
        [        377,         193,         427,         253,     0.66681,           0],
        [        439,         207,         465,         246,     0.62444,           0]]], dtype=float32)

In [ ]:
 # 인식된 얼굴에 대한 후속 처리
image = cv2.imread(image_path)
height, width, channels = image.shape
# print(image.shape)

indexes = int(np_array.shape[1])
# print('indexes :', indexes)

def blur():
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
#         print(out)

        for detection in out:
#             scores = detection[4:]
#             class_id = np.argmax(scores)
#             confidence = scores[class_id]

            x = int(detection[0])
            y = int(detection[1])
            w = int(detection[2])
            h = int(detection[3])


            boxes.append([x, y, w, h])
#             confidences.append(float(confidence))
#             class_ids.append(class_id)
#             print(x,y,w,h)


    # 얼굴 경계 상자 비쥬얼라이제이션
#     indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.9)


#     font = cv2.FONT_HERSHEY_SIMPLEX
    blur_img = cv2.GaussianBlur(image ,(99,99), 30)
#     print(len(boxes))
    for i in range(indexes):
        if i in range(len(boxes)):
            x, y, w, h = boxes[i]
#             print(boxes[i])
            # label = f'{classes[class_ids[i]]}: {confidences[i]:.2f}'
            face_image = blur_img[y:h, x:w]
            image[y:h, x:w] = face_image

    save_path = 'C:/Users/pc3/Desktop/unidentify_10000ea/' + '1682576861142292976.png'
    cv2.imwrite(save_path, image)

# all of files in folder
- check input file directory & output file directory

In [ ]:
#input, output 경로 정하기
input_path = 'C:/Users/pc3/Desktop/unidentify_10000ea/KAMO/ident_image/front/'
output_path = 'C:/Users/pc3/Desktop/unidentify_10000ea/KAMO_un/front_/'

In [76]:
def blur(img):
    image_path = input_path + img
    outs = model.predict(
                    source=image_path,
                    conf=0.1,
                    save=False,
                    imgsz=1280,
                    hide_labels=False,
                    max_det=1000,
                    show=False,
                    )

    np_array = np.array([tensor.numpy() for tensor in outs])

     # 인식된 얼굴에 대한 후속 처리
    image = cv2.imread(image_path)
    height, width, channels = image.shape

    indexes = int(np_array.shape[1])
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:

        for detection in out:

            x = int(detection[0])
            y = int(detection[1])
            w = int(detection[2]) #x2
            h = int(detection[3]) #y2

            boxes.append([x, y, w, h])

    blur_img = cv2.GaussianBlur(image ,(99,99), 30)

    for i in range(indexes):
        if i in range(len(boxes)):
            x, y, w, h = boxes[i]

            face_image = blur_img[y:h, x:w]
            image[y:h, x:w] = face_image

    save_path = output_path + img
    cv2.imwrite(save_path, image)

In [77]:
file_list = os.listdir(input_path)
len(file_list)

2500

In [78]:
for i in file_list:
    blur(i)

Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400100790301085.png: 736x1280 1 face, 769.9ms
Speed: 8.0ms pre-process, 769.9ms inference, 25.3ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400103957161665.png: 736x1280 1 face, 565.0ms
Speed: 7.0ms pre-process, 565.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400104790544748.png: 736x1280 2 faces, 703.0ms
Speed: 

Speed: 5.0ms pre-process, 483.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400306606078386.png: 736x1280 2 faces, 485.0ms
Speed: 6.0ms pre-process, 485.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400324274212360.png: 736x1280 1 face, 521.0ms
Speed: 4.0ms pre-process, 521.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\

image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400453617654800.png: 736x1280 1 face, 465.0ms
Speed: 5.0ms pre-process, 465.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400462784835100.png: 736x1280 2 faces, 499.0ms
Speed: 5.0ms pre-process, 499.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400468785424709.png: 736x1280 544.0ms
Speed: 5.0ms pre-process, 544.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusi

Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400656933317900.png: 736x1280 1 face, 490.0ms
Speed: 8.0ms pre-process, 490.0ms inference, 3.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400660300279617.png: 736x1280 502.0ms
Speed: 5.0ms pre-process, 502.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400664167224646.png: 736x1280 1 face, 595.0ms
Speed: 5.0ms pre-

Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400875117258072.png: 736x1280 591.0ms
Speed: 7.0ms pre-process, 591.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400875283930302.png: 736x1280 485.0ms
Speed: 6.0ms pre-process, 485.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1682400877283954382.png: 736x1280 554.0ms
Speed: 6.0ms pre-process, 554.0ms

...

image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684904777686951876.png: 736x1280 513.0ms
Speed: 6.0ms pre-process, 513.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684904785192692757.png: 736x1280 488.0ms
Speed: 4.0ms pre-process, 488.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684904791030238152.png: 736x1280 473.0ms
Speed: 4.0ms pre-process, 473.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Mod

image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684904983138420105.png: 736x1280 1 face, 524.0ms
Speed: 8.0ms pre-process, 524.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684904985473511934.png: 736x1280 498.0ms
Speed: 4.0ms pre-process, 498.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684904992178462982.png: 736x1280 526.0ms
Speed: 6.0ms pre-process, 526.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers

image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684905976270818710.png: 736x1280 484.0ms
Speed: 4.0ms pre-process, 484.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684905988947097301.png: 736x1280 1 face, 547.0ms
Speed: 5.0ms pre-process, 547.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684905989614247084.png: 736x1280 1 face, 479.0ms
Speed: 5.0ms pre-process, 479.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusin

Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684906517736697912.png: 736x1280 1 face, 535.0ms
Speed: 5.0ms pre-process, 535.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684906526910256147.png: 736x1280 1 face, 473.0ms
Speed: 6.0ms pre-process, 473.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684906530913200617.png: 736x1280 2 faces, 467.0ms
Speed: 4.0ms pre-process, 467.0ms inference, 2.0ms postprocess per

Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684907320228564024.png: 736x1280 1 face, 496.0ms
Speed: 5.0ms pre-process, 496.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684907329402033091.png: 736x1280 520.0ms
Speed: 4.0ms pre-process, 520.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
Ultralytics YOLOv8.0.0  Python-3.11.3 torch-2.0.1+cpu CPU
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 C:\Users\pc3\Desktop\unidentify_10000ea\NAVYA_un\front_\1684907334739329100.png: 736x1280 477.0ms
Speed: 4.0ms pre-process,

https://187cm.tistory.com/31  
https://github.com/noorkhokhar99/face-detection-yolov8/blob/main/detect/predict.py  
https://jjoh4803.tistory.com/36  
https://docs.ultralytics.com/modes/predict/#inference-sources
https://blog.naver.com/PostView.naver?blogId=skfnsid123&logNo=223000302805&parentCategoryNo=&categoryNo=21&viewDate=&isShowPopularPosts=true&from=search  
https://blog.roboflow.com/whats-new-in-yolov8/  
https://github.com/ultralytics/ultralytics  